# Time Response of Linear Systems

## Contents:

1. [Equations of Motion](#eoms)
1. [Transfer Functions](#tfs)
1. [Step Response](#step)
1. [Comparing Systems, Multi-line Plots](#comp)
1. [scipy.signal.lsim: Response to Arbitrary Inputs](#lsim)

<a id='eoms'></a>

### Equations of Motion

Let's begin with a simple, yet representative, mechanical system: the mass-spring-damper system shown below.  A free-body-diagram (FBD) of this system appears to the right.

<img src="https://github.com/AdaWick/MEEG312/blob/master/Lab1/mass%20spring%20damper.png?raw=true" width="600" />

The equation of motion (EOM) for this system derived using Newton's Second Law is as follows:

\\(m \ddot{x} = \sum F_x = -b \dot{x} - kx + F\\)

which can be rewritten as

\\(m \ddot{x} + b \dot{x} + kx = F\\)

<a id='tfs'></a>

### Transfer Functions

The transfer function of a system is defined by the equation:

\\(G(s) = \dfrac{\mathcal{L}[\text{output}]}{\mathcal{L}[\text{input}]}\\)

where \\(\mathcal{L}[\cdot]\\) signifies the Laplace transform *assuming zero initial conditions*.  Of course, different systems will have different inputs and outputs.  For example, inputs may be force, torque, voltage, pressure, etc.  Outputs may be position, velocity, current, flow rate, etc.

For the mass-spring-damper system, the input is \\(F(t)\\) and the output is \\(x(t)\\).  Hence,

\\(G(s) = \dfrac{\mathcal{L}[x(t)]}{\mathcal{L}[F(t)]} \equiv \dfrac{X(s)}{F(s)}\\)

where we've used the usual convention that the Laplace variable is capatalized if the time variable is not.

Taking the Laplace transform of the EOM (assuming zero initial conditions) yields

\\(ms^2X(s) + bsX(s) + kX(s) = F(s)\\)

(Recall that each time derivative produces an \\(s\\) factor in front of the transformed function.)

Rearranging the last equation into transfer function form gives

\\(G(s) = \dfrac{X(s)}{F(x)} = \dfrac{1}{ms^2 + bs + k}\\)

Now, let's enter this transfer function into Python.  To study linear systems, we will make heavy use of the [scipy.signal package](https://docs.scipy.org/doc/scipy/reference/signal.html), particularly the functions under the "Continuous Time Linear Systems" and "LTI Representations" categories.  Specifically, to create a transfer function, we will use the [scipy.signal.TransferFunction](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.TransferFunction.html) command.  The first input is an array of numerator coefficients, and the second input is an array of denominator coefficients.

In [ ]:
import scipy.signal as ss
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
m = 5
b = 1
k = 10
num = [1]
den = [m, b, k]

G = ss.TransferFunction(num,den)
print(G)

You can see that the output shows two arrays, one for the numerator and one for the denominator.  Python automatically normalizes the transfer function so that the leading coefficient of the denominator is 1.

You can get this result yourself by dividing the numerator and denominator by $m$:

$G(s) = \dfrac{1}{ms^2 + bs + k} = \dfrac{1}{5s^2 + s + 10} = \dfrac{0.2}{s^2 + 0.2s + 2}$

**Exercise:**

In the following code block, create these transfer functions:

1. \\(G_1(s) = \dfrac{s+2}{2s+4}\\)

2. \\(G_2(s) = \dfrac{s+2}{2s^2+4}\\)

3. \\(G_3(s) = \dfrac{s}{s^2+6s+4}\\)

Note that you must use a `0` placeholder if a power of \\(s\\) is unwritten.

<a id='step'></a>

### Step Response

The [step response](https://en.wikipedia.org/wiki/Step_response) of a linear system is the response of a system, initially at rest, to a unit step input.  This type of input is \\(0\\) for \\(t > 0\\) and \\(1\\) for \\(t \geq 0\\).

Some examples of applying a (non-unit) step:

* Dropping a ball: the total external force steps from \\(0\\) to \\(-mg\\).

* Connecting a motor to a battery: the applied voltage steps from \\(0\\) to the battery voltage

The [scipy.signal.step](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.step.html) command takes as its input a transfer function, i.e. the ouput of the `TransferFunction` function.  To plot the response, we need to import [pyplot](https://matplotlib.org/api/pyplot_summary.html).

In [ ]:
t,y = ss.step(G)

plt.plot(t,y)
plt.title('Step response');

**Exercise:**

Plot the unit step responses for the systems \\(G_1(s)\\), \\(G_2(s)\\), and \\(G_3(s)\\) from the previous exercise.  How do these responses differ qualitatively?

Let's say you want to compute the step response at specific points in time.  First, let's create our own time vector, increments of 0.5 seconds from 0 to 70 seconds.  Then, we can use this vector as the second input to the [scipy.signal.step](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.step.html) function.

To create this time vector, we will use the [numpy.arange](https://docs.scipy.org/doc/numpy/reference/generated/numpy.arange.html) function, which creates an array with a given starting point, stopping point, and step size.

In [ ]:
t,y = ss.step(G,T=np.arange(0,70.5,0.5))

plt.plot(t,y)
plt.title('Step response');

Note that this plot still looks a bit rough.  Let's decrease the step size to 0.1 and try again.

In [ ]:
t,y = ss.step(G,T=np.arange(0,70.1,0.1))

plt.plot(t,y)
plt.title('Step response');

This plot looks a lot smoother.

<a id='comp'></a>

### Comparing Systems, Multi-line Plots

When designing the dynamic response of a system, you may wish to plot the step responses of several systems using different parameter values.  The step response of the mass-spring-damper system plotted previously had many high frequency oscillations.  Perhaps different value of spring constant \\(k\\) could reduce or eliminate those oscillations.

Let's plot the step response for the mass-spring-damper system for several values of \\(k\\), all on the same axes.

In [ ]:
m = 5
b = 1
k_list = [5, 10, 12]   # list of different k values to plot

for k in k_list:
    num = [1]
    den = [m, b, k]
    G = ss.TransferFunction(num,den)
    
    t,y = ss.step(G,T=np.arange(0,70.1,0.1))
    plt.plot(t,y,label = f'k = {k}') # replaces {} with the value of k

plt.legend();

<a id='lsim'></a>

### scipy.signal.lsim: Response to Arbitrary Inputs

So far we've plotted the response of the mass-spring-damper system to a unit step input using the function [scipy.signal.step](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.step.html).

You can similarly plot the [unit impulse response](https://en.wikipedia.org/wiki/Impulse_response) of a linear system using the [scipy.signal.impulse](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.impulse.html) function.

In this section, you will utilize a more genertic function for plotting the response of a linear system to any arbitrary function of time: [scipy.signal.lsim](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.lsim.html).

To use this function, we need to specify an array of time steps \\(t\\)  over which the system will be simulated and an array of values of the input \\(u(t)\\) at these time steps.

In this example, we will plot the response of the system to the input \\(u(t) = 3 \sin(2t)\\):

In [ ]:
t = np.linspace(0,40,201)
u = 3*np.sin(2*t)

t_out,y,x = ss.lsim(G,u,t)

plt.plot(t,u,t_out,y)
plt.legend(('Input u(t)','Output y(t)'));

Note that the system seems to reach steady state after \\(t=30\\).